In [ ]:
# import used libraries 
import pandas as pd 
import numpy as np
import plotly.express as px 

import sys
from pathlib import Path

import nbformat


In [ ]:
# import paths 
PROJECT_ROOT = Path.cwd().parent  # letterboxd-stats
sys.path.insert(0, str(PROJECT_ROOT))

from config.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR
from scraping_functions import *
from api_key_omdb import OMDB_API_KEY

In [ ]:
diary_df = pd.read_csv(RAW_DATA_DIR.joinpath('diary.csv'))

In [ ]:
# make it easier for work with columns by renaming them to lowercase and replacing spaces with underscores
diary_df.columns = diary_df.columns.str.lower().str.replace(' ', '_')

# format columns correctly 
# date and watched_date to datetime
diary_df['date'] = pd.to_datetime(diary_df['date'], format='%Y-%m-%d')
diary_df['watched_date'] = pd.to_datetime(diary_df['watched_date'], format='%Y-%m-%d')
# rewatch to boolean
diary_df.loc[diary_df['rewatch'].isna(), 'rewatch'] = False
diary_df.loc[diary_df['rewatch']=='Yes', 'rewatch'] = True
diary_df['rewatch'] = diary_df['rewatch'].astype(bool)

# tags has a list of tags separated by commas
diary_df['tags'] = diary_df['tags'].str.replace(', ', ',')

diary_df['tags'] = diary_df['tags'].str.split(',')
# unique identifier for movies
# we'll say that a movie with the same name and the same year is the same movie since there's no other distinguishing feature
diary_df['unique_movies'] = diary_df['name'] + ' (' + diary_df['year'].astype(str) + ')'


# get more film data from omdb 

In [ ]:
#-------- comment back in to get more information from letterboxd (e.g. imdb link, actors, crew,...) ---------
# # in order to 
# urls = diary_df['letterboxd_uri'].tolist()
# diary_df['letterboxd_url_long'] = await resolve_all(urls)

# # --- optional: check how many failed ---
# failed = diary_df['letterboxd_url_long'].isna().sum()
# print(f"{failed} URLs could not be resolved.")

In [ ]:
# test with less entries
diary_df = diary_df.sample(n=30, random_state=1)

In [ ]:
for idx, row in diary_df.iterrows():
    title = row['name']  
    year = row['year']   

    if not title or not year:
        continue

    omdb_data = get_omdb_info(title, year, OMDB_API_KEY)

    if omdb_data:
        for key, value in omdb_data.items():
            col_name = f"omdb_{key}"
            # Falls value Liste oder Dict, in String umwandeln
            if isinstance(value, (list, dict)):
                value = str(value)
            diary_df.loc[idx, col_name] = value
    else:
        print(f"Movie not found on OMDb: {title} ({year})")


# all time watches per movie

In [ ]:
watches = diary_df.value_counts(['unique_movies']).reset_index(name='watch_count_per_movie').sort_values('watch_count_per_movie', ascending=False)

watch_count_per_movie = watches['watch_count_per_movie'].value_counts().sort_index(ascending=False).reset_index(name='watches')

fig = px.pie(watch_count_per_movie, 
             values='watches', 
             names=watch_count_per_movie.index, 
             title='Distribution of Watch Counts per Movie')

fig.update_layout(
    legend=dict(
        title=dict(text='Watches per Movie') 
    )
)

fig.update_layout(
    annotations=[
        dict(
            text='all time in diary',  
            x=-0.15,  
            y=1.175, 
            xref='paper',
            yref='paper',
            showarrow=False,
            font=dict(size=10)
        )
    ]
)

fig.show()

# top 10 movies

In [ ]:
top10_movies = watches.head(10)

# if there's not 10 movies with multiple watches, show less than 10 movies 
top10_movies = top10_movies[top10_movies['watch_count_per_movie']>1]
top_x_movies = len(top10_movies)

fig = px.bar(top10_movies, 
             x='unique_movies', 
             y='watch_count_per_movie', 
             title=f'Top {top_x_movies} Most Watched Movies with more than 1 watch',
             labels={'name': 'Movie Name', 'watch_count_per_movie': 'Number of Watches'})


fig.show()

In [ ]:
# ratings of top x movies 
top10_movies_list = list(top10_movies['unique_movies'])

top_x_movies_ratings = diary_df[diary_df['unique_movies'].isin(top10_movies_list)]
top_x_movies_ratings

fig = px.scatter(top_x_movies_ratings, 
             x='watched_date', 
             y='rating', 
             color='unique_movies',
             title=f'Ratings for Top {top_x_movies} Most Watched Movies with more than 1 watch',
             labels={'name': 'Movie Name', 'watch_count_per_movie': 'Number of Watches'})
fig.show()

In [ ]:
# movies per day 
movies_per_day = diary_df['watched_date'].value_counts().sort_index()
fig = px.line(movies_per_day, 
              x=movies_per_day.index, 
              y=movies_per_day.values, 
              title='Movies Watched per Day',
              labels={'x': 'Watch Date', 'y': 'Number of Movies Watched'})
fig.show()

In [ ]:
# top 10 dates with the most movies watched 
top10_dates = movies_per_day.sort_values(ascending=False).head(10).reset_index()
top10_dates.columns = ['watched_date', 'number_of_movies_watched']
top10_dates_fig = px.bar(top10_dates, 
                    x='watched_date', 
                    y='number_of_movies_watched',
                    title='Top 10 Dates with Most Movies Watched',
                    labels={'watched_date': 'Watch Date', 'number_of_movies_watched': 'Number of Movies Watched'})
top10_dates_fig.show()

In [ ]:
# rewatch percentage 
rewatches = diary_df.value_counts('rewatch').reset_index(name='rewatch_count')
fig = px.pie(rewatches, 
    values='rewatch_count', 
    names='rewatch',
    title='Rewatch vs. First-time Watch Distribution')

fig.update_layout(
    legend=dict(
        title=dict(text='Rewatch') 
    )
)

fig.show()

In [ ]:
used_tags = diary_df['tags'].explode().value_counts().reset_index(name='tag_count')
fig = px.bar(used_tags, 
       x='tags', 
       y='tag_count',
       title='Used tags count',
       labels={'tags': 'Tag', 'tag_count': 'Count'})

fig.show()

In [ ]:
diary_df['omdb_Genre_list'] = diary_df['omdb_Genre'].str.split(', ')
genres = diary_df['omdb_Genre_list'].explode().value_counts().reset_index(name='genre_count')
genres.rename(columns={'omdb_Genre_list': 'genre'}, inplace=True)
fig = px.bar(genres, 
       x='genre', 
       y='genre_count',
       title='Genre distribution',
       labels={'index': 'Genre', 'genre_count': 'Count'})

fig.show()

# Average rating for release years 

In [ ]:
year_stats = (
    diary_df
    .groupby('year')
    .agg(
        avg_rating=('rating', 'mean'),
        movie_count=('rating', 'count')
    )
    .reset_index()
)

best_years = year_stats.sort_values('avg_rating', ascending=False).head(3)
worst_years = year_stats.sort_values('avg_rating').head(3)

overall_avg = diary_df['rating'].mean()

year_stats['category'] = year_stats['avg_rating'].apply(
    lambda x: 'Favorite year' if x > overall_avg else 'Least favorite year'
)

fig = px.scatter(
    year_stats,
    x='year',
    y='avg_rating',
    size='movie_count',
    color='avg_rating',
    color_continuous_scale='RdYlGn',
    title='Average Film Rating per Year',
    labels={
        'year': 'Year',
        'avg_rating': 'Ø rating',
        'movie_count': 'Movie count'
    }
)

fig.add_hline(
    y=overall_avg,
    line_dash='dash',
    annotation_text='Average Rating',
    annotation_position='bottom right'
)

fig.show()
